In [1]:
import numpy as np
import torch
from torchinfo import summary
from backbones import get_model

# Init and load training model

In [2]:
model_training = get_model('repvgg', dropout=0.0, fp16=False, num_features=512)

In [3]:
# Load RepVGG training checkpoint
model_training.load_state_dict(torch.load('work_dirs/ms1mv3_repvgg/model.pt', map_location=torch.device('cpu')), strict=True)
model_training = model_training.eval()

In [4]:
x = torch.rand((1, 3, 112, 112))
with torch.no_grad():
    y_training = model_training(x)

# Init and load deploy model

In [5]:
model_deploy = get_model('repvgg', dropout=0.0, fp16=False, num_features=512)
for m in model_deploy.modules():
    if hasattr(m, 'switch_to_deploy'):
        m.switch_to_deploy()

In [6]:
model_deploy.load_state_dict(torch.load('work_dirs/ms1mv3_repvgg/model_deploy.pt', map_location=torch.device('cpu')), strict=True)
model_deploy = model_deploy.eval()

In [7]:
with torch.no_grad():
    y_deploy = model_deploy(x)

# Check output

In [8]:
torch.allclose(y_training, y_deploy, rtol=1e-5, atol=1e-5)

True

In [9]:
euclidena_dist = sum(((y_deploy - y_training)**2).reshape(512))
print('Euclidean distance:', euclidena_dist.item())

Euclidean distance: 9.242296927780558e-10


In [10]:
cos = torch.nn.CosineSimilarity(dim=1)(y_training, y_deploy)
print('Cosine similarity:', cos.item())

Cosine similarity: 1.0
